<a href="https://colab.research.google.com/github/Syeda-Eman/Machine_Learning/blob/main/Developer_Stress_Level_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

developer_stress_level_prediction_path = kagglehub.competition_download('developer-stress-level-prediction')

print('Data source import complete.')


# Importing libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

# Loading data

In [ ]:
train = pd.read_csv('/kaggle/input/developer-stress-level-prediction/train.csv')
test  = pd.read_csv('/kaggle/input/developer-stress-level-prediction/test.csv')
submission = pd.read_csv('/kaggle/input/developer-stress-level-prediction/submission.csv')

# Exploring data

In [ ]:
train.head()

,ID,Hours_Worked,Sleep_Hours,Bugs,Deadline_Days,Coffee_Cups,Meetings,Interruptions,Experience_Years,Code_Complexity,Remote_Work,Stress_Level
0,249,9,6,36,2,6,15,9,Junior,High,No,100.000000
1,433,13,3,39,25,5,5,2,Senior,Medium,Yes,100.000000
2,19,5,4,50,58,1,1,2,Senior,High,No,100.000000
3,322,5,5,39,4,8,1,8,Junior,Medium,Yes,99.504878
4,332,11,4,48,2,2,17,2,Junior,Low,No,100.000000


In [ ]:
test.head()

,ID,Hours_Worked,Sleep_Hours,Bugs,Deadline_Days,Coffee_Cups,Meetings,Interruptions,Experience_Years,Code_Complexity,Remote_Work
0,361,4,4,7,59,9,1,3,Junior,Medium,Yes
1,73,9,4,9,53,0,11,9,Senior,Medium,No
2,374,6,3,13,42,1,18,1,Senior,Low,No
3,155,7,7,41,34,3,17,2,Mid,Low,Yes
4,104,4,6,39,50,3,5,8,Junior,Medium,No


In [ ]:
submission.head()

,ID,Stress_Level
0,361,72.032023
1,73,5.081853
2,374,40.303965
3,155,23.897284
4,104,74.105704


In [ ]:
train.shape

(400, 12)

In [ ]:
test.shape

(100, 11)

In [ ]:
submission.shape

(100, 2)

# Model training

In [ ]:
X = train.drop(columns='Stress_Level')
y = train['Stress_Level']
X_test = test

# Identify categorical and numeric features
cat_cols = ['Experience_Years', 'Code_Complexity', 'Remote_Work']
num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
num_cols = [c for c in num_cols if c not in ['Stress_Level']]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# 3a. Tree-based pipeline (no scaling needed)
tree_preprocessor = ColumnTransformer([
    ('num', 'passthrough', num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

# 3b. Linear / distance-based pipeline (scaling numeric features)
linear_preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

In [ ]:
models = {
    'LinearRegression': Pipeline([
        ('preprocessor', linear_preprocessor),
        ('regressor', LinearRegression())
    ]),

    'Ridge': Pipeline([
        ('preprocessor', linear_preprocessor),
        ('regressor', Ridge(alpha=1.0))
    ]),

    'Lasso': Pipeline([
        ('preprocessor', linear_preprocessor),
        ('regressor', Lasso(alpha=0.01))
    ]),

    'RandomForest': Pipeline([
        ('preprocessor', tree_preprocessor),
        ('regressor', RandomForestRegressor(
            n_estimators=200, max_depth=5, random_state=42
        ))
    ]),

    'GradientBoosting': Pipeline([
        ('preprocessor', tree_preprocessor),
        ('regressor', GradientBoostingRegressor(
            n_estimators=300, learning_rate=0.05, max_depth=3, random_state=42
        ))
    ])
}

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for name, model in models.items():
    rmses = []
    for train_idx, val_idx in kf.split(X):
        X_tr, X_va = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_va = y.iloc[train_idx], y.iloc[val_idx]

        model.fit(X_tr, y_tr)
        preds = model.predict(X_va)
        rmse = np.sqrt(mean_squared_error(y_va, preds))
        rmses.append(rmse)

    print(f"{name}: {np.mean(rmses):.4f}")

LinearRegression: 16.3935
Ridge: 16.3895
Lasso: 16.3896
RandomForest: 10.0579
GradientBoosting: 8.1830


In [ ]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

In [ ]:
x_pred = model.predict(X_train_scaled)

In [ ]:
rmse_value = root_mean_squared_error(X_test_scaled, x_pred)
print(f"RMSE: {rmse_value}")

# Task
Train the GradientBoosting model on the full training dataset, use it to predict 'Stress_Level' for the test dataset, and generate a submission file in the format (ID, Stress_Level).

## Train Best Model

### Subtask:
Train the GradientBoosting model on the full training dataset (X and y) as it showed the lowest RMSE during K-fold cross-validation.


**Reasoning**:
Access the 'GradientBoosting' model from the `models` dictionary and train it on the full training dataset (X, y) as instructed.



In [1]:
best_model = models['GradientBoosting']
best_model.fit(X, y)
print("GradientBoosting model trained successfully on the full dataset.")

NameError: name 'models' is not defined

**Reasoning**:
The previous cell failed because the `models` dictionary was not defined in the current execution context. To fix this, I need to include the definitions of `X`, `y`, `cat_cols`, `num_cols`, `tree_preprocessor`, `linear_preprocessor`, and the `models` dictionary itself from previous successfully executed cells, before attempting to access and train the 'GradientBoosting' model.



In [2]:
X = train.drop(columns='Stress_Level')
y = train['Stress_Level']

# Identify categorical and numeric features (from cell 9Vj6ziiE5oNN)
cat_cols = ['Experience_Years', 'Code_Complexity', 'Remote_Work']
num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
# Ensure 'ID' is removed from numeric columns as it's an identifier
if 'ID' in num_cols:
    num_cols.remove('ID')

# Define preprocessors (from cell aKG2mj0s5oNO)
tree_preprocessor = ColumnTransformer([
    ('num', 'passthrough', num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

linear_preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

# Define models dictionary (from cell l5tGY7tH5oNO)
models = {
    'LinearRegression': Pipeline([
        ('preprocessor', linear_preprocessor),
        ('regressor', LinearRegression())
    ]),

    'Ridge': Pipeline([
        ('preprocessor', linear_preprocessor),
        ('regressor', Ridge(alpha=1.0))
    ]),

    'Lasso': Pipeline([
        ('preprocessor', linear_preprocessor),
        ('regressor', Lasso(alpha=0.01))
    ]),

    'RandomForest': Pipeline([
        ('preprocessor', tree_preprocessor),
        ('regressor', RandomForestRegressor(
            n_estimators=200, max_depth=5, random_state=42
        ))
    ]),

    'GradientBoosting': Pipeline([
        ('preprocessor', tree_preprocessor),
        ('regressor', GradientBoostingRegressor(
            n_estimators=300, learning_rate=0.05, max_depth=3, random_state=42
        ))
    ])
}

# Train the GradientBoosting model on the full training dataset
best_model = models['GradientBoosting']
best_model.fit(X, y)
print("GradientBoosting model trained successfully on the full dataset.")

NameError: name 'train' is not defined

**Reasoning**:
The previous cell failed because the `train` DataFrame was not defined. I need to ensure that the `train` DataFrame is loaded before it is used to create `X` and `y`, and also load the `test` DataFrame as it will be needed later for predictions. I will include the data loading step at the beginning of the code block.



In [3]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Load the dataframes (from cell L428AioC5oNG)
train = pd.read_csv('/kaggle/input/developer-stress-level-prediction/train.csv')
test = pd.read_csv('/kaggle/input/developer-stress-level-prediction/test.csv')

X = train.drop(columns='Stress_Level')
y = train['Stress_Level']
X_test = test # Assign X_test as test for later use

# Identify categorical and numeric features (from cell 9Vj6ziiE5oNN)
cat_cols = ['Experience_Years', 'Code_Complexity', 'Remote_Work']
num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
# Ensure 'ID' is removed from numeric columns as it's an identifier and not a feature
if 'ID' in num_cols:
    num_cols.remove('ID')

# Define preprocessors (from cell aKG2mj0s5oNO)
tree_preprocessor = ColumnTransformer([
    ('num', 'passthrough', num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

linear_preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

# Define models dictionary (from cell l5tGY7tH5oNO)
models = {
    'LinearRegression': Pipeline([
        ('preprocessor', linear_preprocessor),
        ('regressor', LinearRegression())
    ]),

    'Ridge': Pipeline([
        ('preprocessor', linear_preprocessor),
        ('regressor', Ridge(alpha=1.0))
    ]),

    'Lasso': Pipeline([
        ('preprocessor', linear_preprocessor),
        ('regressor', Lasso(alpha=0.01))
    ]),

    'RandomForest': Pipeline([
        ('preprocessor', tree_preprocessor),
        ('regressor', RandomForestRegressor(
            n_estimators=200, max_depth=5, random_state=42
        ))
    ]),

    'GradientBoosting': Pipeline([
        ('preprocessor', tree_preprocessor),
        ('regressor', GradientBoostingRegressor(
            n_estimators=300, learning_rate=0.05, max_depth=3, random_state=42
        ))
    ])
}

# Train the GradientBoosting model on the full training dataset
best_model = models['GradientBoosting']
best_model.fit(X, y)
print("GradientBoosting model trained successfully on the full dataset.")

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/developer-stress-level-prediction/train.csv'

**Reasoning**:
The previous cell failed with a `FileNotFoundError` because the data files were not found at the hardcoded `/kaggle/input/` path. This environment uses `kagglehub.competition_download` to access data, which returns the path to the downloaded files. I need to ensure `kagglehub.competition_download` is called and then use the returned path (`developer_stress_level_prediction_path`) to correctly load the `train.csv` and `test.csv` files.



In [4]:
import pandas as pd
import os
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Ensure kagglehub is initialized and data is downloaded
import kagglehub
# Assuming login is already handled by previous cell 8v7s5blm5oM8 if it were executed
developer_stress_level_prediction_path = kagglehub.competition_download('developer-stress-level-prediction')

# Load the dataframes using the path from kagglehub
train = pd.read_csv(os.path.join(developer_stress_level_prediction_path, 'train.csv'))
test = pd.read_csv(os.path.join(developer_stress_level_prediction_path, 'test.csv'))

X = train.drop(columns='Stress_Level')
y = train['Stress_Level']
X_test = test # Assign X_test as test for later use

# Identify categorical and numeric features
cat_cols = ['Experience_Years', 'Code_Complexity', 'Remote_Work']
num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
# Ensure 'ID' is removed from numeric columns as it's an identifier and not a feature
if 'ID' in num_cols:
    num_cols.remove('ID')

# Define preprocessors
tree_preprocessor = ColumnTransformer([
    ('num', 'passthrough', num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

linear_preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

# Define models dictionary
models = {
    'LinearRegression': Pipeline([
        ('preprocessor', linear_preprocessor),
        ('regressor', LinearRegression())
    ]),

    'Ridge': Pipeline([
        ('preprocessor', linear_preprocessor),
        ('regressor', Ridge(alpha=1.0))
    ]),

    'Lasso': Pipeline([
        ('preprocessor', linear_preprocessor),
        ('regressor', Lasso(alpha=0.01))
    ]),

    'RandomForest': Pipeline([
        ('preprocessor', tree_preprocessor),
        ('regressor', RandomForestRegressor(
            n_estimators=200, max_depth=5, random_state=42
        ))
    ]),

    'GradientBoosting': Pipeline([
        ('preprocessor', tree_preprocessor),
        ('regressor', GradientBoostingRegressor(
            n_estimators=300, learning_rate=0.05, max_depth=3, random_state=42
        ))
    ])
}

# Train the GradientBoosting model on the full training dataset
best_model = models['GradientBoosting']
best_model.fit(X, y)
print("GradientBoosting model trained successfully on the full dataset.")

UnauthenticatedError: User is not authenticated

**Reasoning**:
The previous cell failed with an `UnauthenticatedError` because `kagglehub.login()` was not executed, which is necessary before downloading competition data. I will explicitly call `kagglehub.login()` at the beginning of the code block to ensure proper authentication.



In [5]:
import pandas as pd
import os
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Ensure kagglehub is initialized and data is downloaded
import kagglehub
kagglehub.login() # Explicitly call login to handle UnauthenticatedError
developer_stress_level_prediction_path = kagglehub.competition_download('developer-stress-level-prediction')

# Load the dataframes using the path from kagglehub
train = pd.read_csv(os.path.join(developer_stress_level_prediction_path, 'train.csv'))
test = pd.read_csv(os.path.join(developer_stress_level_prediction_path, 'test.csv'))

X = train.drop(columns='Stress_Level')
y = train['Stress_Level']
X_test = test # Assign X_test as test for later use

# Identify categorical and numeric features
cat_cols = ['Experience_Years', 'Code_Complexity', 'Remote_Work']
num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
# Ensure 'ID' is removed from numeric columns as it's an identifier and not a feature
if 'ID' in num_cols:
    num_cols.remove('ID')

# Define preprocessors
tree_preprocessor = ColumnTransformer([
    ('num', 'passthrough', num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

linear_preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

# Define models dictionary
models = {
    'LinearRegression': Pipeline([
        ('preprocessor', linear_preprocessor),
        ('regressor', LinearRegression())
    ]),

    'Ridge': Pipeline([
        ('preprocessor', linear_preprocessor),
        ('regressor', Ridge(alpha=1.0))
    ]),

    'Lasso': Pipeline([
        ('preprocessor', linear_preprocessor),
        ('regressor', Lasso(alpha=0.01))
    ]),

    'RandomForest': Pipeline([
        ('preprocessor', tree_preprocessor),
        ('regressor', RandomForestRegressor(
            n_estimators=200, max_depth=5, random_state=42
        ))
    ]),

    'GradientBoosting': Pipeline([
        ('preprocessor', tree_preprocessor),
        ('regressor', GradientBoostingRegressor(
            n_estimators=300, learning_rate=0.05, max_depth=3, random_state=42
        ))
    ])
}

# Train the GradientBoosting model on the full training dataset
best_model = models['GradientBoosting']
best_model.fit(X, y)
print("GradientBoosting model trained successfully on the full dataset.")

UnauthenticatedError: User is not authenticated